<a href="https://colab.research.google.com/github/lucasgneccoh/BDSS_Dauphine/blob/main/notebooks/solutions/BDSS_2021_TD6_JSON_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bases de données semi-structurées - TD 6 - JSON, JSONSchema and JSONPath

Main teacher: **Dario COLAZZO**

Teaching Assistant: **Lucas GNECCO**

Special thanks to **Beatrice NAPOLITANO**

Université Paris Dauphine - PSL

# Introduction

Welcome!

In this notebook we will use the `json` and `jsonschema` Python libraries to load, save and validate JSON files and objects. We will also cover some JSONPath using the library `jsonpath-ng`

Here is some important documentation and resources

https://json-schema.org/learn/getting-started-step-by-step.html#starting

# Examples

## `json` library examples
Here we show how to read and write JSON files using the `json` library.

Example file is taken from 

https://json-schema.org/learn/getting-started-step-by-step.html#properties-deeper

In [ ]:
'''
    Create a JSON object and write it in a file
'''

import json

# Take the json example in Python format (a dictionary)
# In Python, the equivalent of a JSON object is a dict
# A JSON array is a list
json_object = {
                "productId": 1,
                "productName": "An ice sculpture",
                "price": 12.50,
                "tags": [ "cold", "ice" ],
                "dimensions": {
                    "length": 7.0,
                    "width": 12.0,
                    "height": 9.5
                    },
                "warehouseLocation": {
                    "latitude": -78.75,
                    "longitude": 20.4
                    }
                }


In [ ]:
'''
    First lets print the object as a string in JSON format
    Then read a JSON object from a string
    Use functions json.loads and json.dumps

'''
s = json.dumps(json_object)
print("JSON object as a string:\n", type(s), s)

json_object = None
json_object = json.loads(s)
print("JSON object read from a string:\n", type(json_object), json_object)


In [ ]:
'''
    Lets write this to a file called myjson.json
    Use function json.dump
'''

filename = "myjson.json"
with open(filename, "w") as f:
    json.dump(json_object, f)
    f.close()

In [ ]:
'''
    Now see the content of the file as it is
'''
!cat myjson.json

In [ ]:
'''
    Lets read it in a Python object to see what it gives 
    Use function json.load
'''
json_object = None
with open(filename, "r") as f:
    json_object = json.load(f)


print(f"The json_object variable is of type: {type(json_object)}")
print()
print(json_object)

## `jsonschema` library
This library allows to validate JSON objects against a JSONSchema

Again we folow the example given in 

https://json-schema.org/learn/getting-started-step-by-step.html#properties-deeper


`jsonschema` documentation available at

https://python-jsonschema.readthedocs.io/en/stable/



In [ ]:
%%capture
!pip install jsonschema
from jsonschema import Draft7Validator, RefResolver

In [ ]:
schemaProducts =  {
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "example.com.product.schema.json",
  "title": "Product",
  "description": "A product from Acme's catalog",
  "type": "object",
  "properties": {
    "productId": {
      "description": "The unique identifier for a product",
      "type": "integer"
    },
    "productName": {
      "description": "Name of the product",
      "type": "string"
    },
    "price": {
      "description": "The price of the product",
      "type": "number",
      "exclusiveMinimum": 0
    },
    "tags": {
      "description": "Tags for the product",
      "type": "array",
      "items": {
        "type": "string"
      },
      "minItems": 1,
      "uniqueItems": True
    },
    "dimensions": {
      "type": "object",
      "properties": {
        "length": {
          "type": "number"
        },
        "width": {
          "type": "number"
        },
        "height": {
          "type": "number"
        }
      },
      "required": [ "length", "width", "height" ]
    },
    "warehouseLocation": {
      "description": "Coordinates of the warehouse where the product is located.",
      "$ref": "example.com.geographical-location.schema.json"
    }
  },
  "required": [ "productId", "productName", "price" ]
}

schemaGeo = {
  "$id": "example.com.geographical-location.schema.json",
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "Longitude and Latitude",
  "description": "A geographical coordinate on a planet (most commonly Earth).",
  "required": [ "latitude", "longitude" ],
  "type": "object",
  "properties": {
    "latitude": {
      "type": "number",
      "minimum": -90,
      "maximum": 90
    },
    "longitude": {
      "type": "number",
      "minimum": -180,
      "maximum": 180
    }
  }
}

json.dump(schemaProducts, open("schemaProducts.json", "w") )
json.dump(schemaGeo, open("schemaGeo.json", "w") )

In [ ]:
'''
    Let's see if the example json object we worked with follows the schema or not
'''
schemaProducts, schemaGeo, json_object = None, None, None
json_object = json.load(open("myjson.json"))
schemaProducts = json.load(open("schemaProducts.json"))
schemaGeo = json.load(open("schemaGeo.json"))


schema_store = {
    schemaProducts['$id'] : schemaProducts,
    schemaGeo['$id'] : schemaGeo,
}

# Create a resolver to work with local files and avoid fetching URLs
resolver = RefResolver.from_schema(schemaProducts, store=schema_store)

validator = Draft7Validator(schemaProducts, resolver=resolver)

try:
    validator.validate(json_object)
    print("File is valid!")
except Exception as e:
    print("File was not validated correctly. Here are the catched exceptions\n")
    print(e)

In [ ]:
'''
    Let's see what happens when we do not follow the schema
'''

json_object['warehouseLocation']['latitude'] = 120
json_object['productId'] = 'Hello'

try:
    validator.validate(json_object)
    print("File is valid!")
except Exception as e:
    print("File was not validated correctly. Here are the catched exceptions\n")
    print(e)

# Excercise 1: Write a schema

To begin with, we are going to create a schema for a very simple situation we already know well... films!

Take the original DTD for the FILMS database and make it a JSON schema. 

**You can change some things if you want to make it more complex or realistic, as long as the Schema is well written and makes sense.**

***NOTE: Be careful, there might be some structural changes needed***

Here is the DTD

```
    <!ELEMENT FILMS (FILM+, ARTISTE+)>
    <!ELEMENT FILM (TITRE, GENRE, PAYS, MES, ROLES, RESUME?)>
    <!ELEMENT TITRE (#PCDATA)>
    <!ATTLIST FILM Annee CDATA #REQUIRED>
    <!ELEMENT GENRE (#PCDATA)>
    <!ELEMENT PAYS (#PCDATA)>
    <!ELEMENT MES (#PCDATA)>
    <!ATTLIST MES id_mes IDREF #IMPLIED>
    <!ELEMENT ROLES (ROLE*)>
    <!ELEMENT ROLE (PRENOM, NOM, INTITULE)>
    <!ELEMENT PRENOM (#PCDATA)>
    <!ELEMENT NOM (#PCDATA)>
    <!ELEMENT INTITULE (#PCDATA)>
    <!ELEMENT RESUME (#PCDATA)>
    <!ELEMENT ARTISTE (ACTNOM, ACTPNOM, ANNEENAISS)>
    <!ATTLIST ARTISTE id_art ID #REQUIRED>
    <!ELEMENT ACTNOM (#PCDATA)>
    <!ELEMENT ACTPNOM (#PCDATA)>
    <!ELEMENT ANNEENAISS (#PCDATA)>
```

In [ ]:
'''
    Get all the genres to create a good schema using SAX
'''
from lxml import etree
import re
from xml.dom.minidom import parse
import xml.sax

class GetTextInsideTag(xml.sax.ContentHandler):
    def __init__(self, tag = None):
        super().__init__()
        self.tag = tag
        self.reading = False
        self.buffer = []
        self.result = []
        

    def startElement(self, name, attrs):
        if not self.tag is None and name == self.tag:
            self.reading = True
        return


    def endElement(self, name):
        if self.buffer:
            # If something was read, then add it to results
            # before reseting the buffer
            self.result.append(' '.join(self.buffer))
        self.reading = False
        self.buffer = []
        return

    def characters(self, content):
        # When text is encountered
        # print("Characters '" + content + "'")
        if self.reading: self.buffer.append(content)
        return



!wget "https://raw.githubusercontent.com/lucasgneccoh/BDSS_Dauphine/main/data/films.xml"
path = "films.xml"


f = open(path)
handler = GetTextInsideTag(tag = "GENRE")
xml.sax.parse(f, handler)
unique_genres = set(handler.result)
print(unique_genres)

f = open(path)
handler = GetTextInsideTag(tag = "PAYS")
xml.sax.parse(f, handler)
unique_pays = set(handler.result)
print(unique_pays)

In [ ]:
'''
    My proposed solution is structured like this:
        - FILMS, the root, is an object containing two arrays: arrFilms and arrArtistes
        - FILM is an object that represents one film
        - ARTISTE is an object that represents one artist
        I made some slight changes to the TITRE element just to show what we can do
'''


schemaRole = {
    "$id": "dauphine.bdss.role",
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "Film role",
    "description": "A role played in a film by an artist.",
    "required": [ "NOM", "PRENOM"],
    "type": "object",
    "properties": {
        "NOM": {
            "description": "Last name of the artist",
            "type": "string",
        },
        "PRENOM": {
            "description": "Name of the artist",
            "type": "string"
        },
        "INTITULE": {
            "description": "Name of the character in the film",
            "type": "string"
        }
    }
}

schemaFilmsObject = {
    "$id": "dauphine.bdss.film",
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "Film object",
    "description": "Object representing one single film.",
    "required": ["Annee", "TITRE", "MES", "ROLES"],
    "type": "object",
    "properties": {
        "Anne": {
            "description": "Integer containing the year of release of the film",
            "type" : "number",
            "minimum" : 1880,
            "maximum" : 2022
        },
        "TITRE": {
            "description": "Object representing the title of the film and making explicit the language and other attributes of the title",
            "type": "object",
            "properties": {
                "title": {
                    "description": "The actual title of the film",
                    "type": "string"
                },
                "lang":{
                    "description": "Language tag representing the language of the title",
                    "type": "string"
                },
                "note":{
                    "description": "Any observation about the title, like its origin, whether it is the official one or not, etc",
                    "type": "string"
                }
            },
            "required": ["title", "lang"]
        },
        "GENRE": {
            "description": "The genre used to tag this film",
            "type": "string",
            "enum": list(unique_genres)
        },
        "PAYS": {
            "description": "The country of the film, whatever that means",
            "type": "string",
            "enum": list(unique_pays)
        },
        "MES": {
            "description": "The artist_id of the director of the film",
            "type": "string"
        },
        "ROLES": {
            "description": "Array of the roles in the movie",
            "type": "array",
            "items": {
                "description": "A role is made of the name and last name of the artist that played the role and the name of the played character",
                "$ref": "dauphine.bdss.role"
            }
        },
        "RESUME": {
            "description": "A short description of the film. make it interesting!",
            "type": "string"
        }
    }
}


schemaArtistObject = {
    "$id": "dauphine.bdss.artiste",
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "Film object",
    "description": "Object representing one single artist.",
    "required": ["id_art", "ACTNOM", "ACTPNOM"],
    "type": "object",
    "properties": {
        "id_art" : {
            "description" : "Unique identifier of the artist in the database",
            "type" : "string"
        },
        "ACTNOM": {
            "description": "Last name of the artist",
            "type": "string"
        },
        "ACTPNOM": {
            "description": "Name of the artist",
            "type": "string"
        },
        "ANNEENAISS": {
            "description": "Name of the character in the film",
            "type": "string"
        }
    }
}



schemaFilmsDatabase = {
    "$id": "dauphine.bdss.filmsDatabase",
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "Our beloved films database",
    "description": "A film database that was once scrapped from the internet.",
    "required": [ "arrFilms", "arrArtistes" ],
    "type": "object",
    "properties": {
        "arrFilms": {
            "description": "The array containing all the films",
            "type": "array",
            "items": {
                "description": "Object of type film",
                "$ref": "dauphine.bdss.film"
            },
            "minItems": 1
        },
        "arrArtistes": {
            "description": "The array containing all the artists",
            "type": "array",
            "items": {
                "description": "Object of type artiste",
                "$ref": "dauphine.bdss.artiste"
            },
            "minItems": 1
        }
    }
}


# Save schemas
json.dump(schemaRole, open('schemaRole.json', 'w'))
json.dump(schemaFilmsObject, open('schemaFilmsObject.json', 'w'))
json.dump(schemaArtistObject, open('schemaArtistObject.json', 'w'))
json.dump(schemaFilmsDatabase, open('schemaFilmsDatabase.json', 'w'))

# Excercise 2: XML to JSON

Transform the original XML Films database into a JSON file (or files) following your schema.

Use your favorite tool: Basic python, SAX, DOM, xml...

Solution using `DOM` from one of the past TDs

In [ ]:
from lxml import etree
import re
from xml.dom.minidom import parse
import copy

dom = parse("films.xml")
filmTextElems = ["TITRE", "GENRE", "PAYS", "RESUME"]
artistTextElems = ["ACTNOM", "ACTPNOM", "ANNEENAISS"]
roleTextElements = ["NOM", "PRENOM", "INTITULE"]

def getText(node):
    try:
        return node.childNodes[0].data
    except Exception as e:  
        print(f'Problems getText with node {node.tagName}')
        raise e

def getAttributes(node):
    res = {}
    if node.hasAttributes():
        for k, v in node.attributes.items():
            res[k] = v
    return res

def getTextElements(node, elements):
    res = {}
    for elem in elements:
        for t in node.getElementsByTagName(elem):
            if t.hasChildNodes():
                res[elem] = getText(t)
            
    return res

# Get films
films = []
for f in dom.getElementsByTagName("FILM"):
    film = getTextElements(f, filmTextElems)
    film.update(getAttributes(f))

    # Read MES
    for m in f.getElementsByTagName('MES'):
        film["MES"] = m.getAttribute('id_mes')
    
    # Read ROLES
    roles = []
    for r in f.getElementsByTagName('ROLE'):
        roles.append(getTextElements(r, roleTextElements))
    
    film.update({'ROLES':  copy.deepcopy(roles)})

    # I created a special TITRE. I have to create it
    film["TITRE"] = {
        "title": film["TITRE"],
        "lang": "@fr",
        "note": "Lorem ipsum"
    }

    films.append(film)


# Get artists
artists = []
for a in dom.getElementsByTagName("ARTISTE"):
    artist = getTextElements(a, artistTextElems)
    artist.update(getAttributes(a))
    artists.append(artist)



FILMS = {'arrArtistes':artists , 'arrFilms':films}

# Save
json.dump(FILMS, open('films.json', 'w'))

# Excercise 3: Validate the data

Validate the FILMS dataset you just created with the JSON schema you created using `jsonschema`

In [ ]:
schema_store = {
    schemaRole['$id']: schemaRole,
    schemaFilmsObject['$id'] : schemaFilmsObject,
    schemaArtistObject['$id'] : schemaArtistObject,
    schemaFilmsDatabase['$id']: schemaFilmsDatabase
}

# Create a resolver to work with local files and avoid fetching URLs
resolver = RefResolver.from_schema(schemaFilmsDatabase, store=schema_store)

validator = Draft7Validator(schemaFilmsDatabase, resolver=resolver)

try:
    validator.validate(FILMS)
    print("File is valid!")
except Exception as e:
    print("File was not validated correctly. Here are the catched exceptions\n")
    print(e)

# Excercise 4: JSONPath to query the dataset

We are going to redo some of the queries we have worked on but using this new tool, **JSONPath**.
The implementation for Python we are going to use is called `jsonpath-ng`.


`jsonpath-ng` PyPi site: https://pypi.org/project/jsonpath-ng/


Some documentation about **JSONPath** comparing it to **XPath**: https://goessner.net/articles/JsonPath/


Here is a nice documentation explaining how JSONPath works, but be careful because the implementation is for Java: https://github.com/json-path/JsonPath


This blog also has some documentation and examples using Python and the library `jsonpath-ng`:
https://blogboard.io/blog/knowledge/jsonpath-python/

In [ ]:
!pip install jsonpath-ng
from jsonpath_ng.ext import parse

## Examples

In [ ]:
'''
    Get all the titles
'''
jsonpath_expr = parse('$.arrFilms[:].TITRE.title')

result = [match.value for match in jsonpath_expr.find(FILMS)]

print("Get all the titles")

print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")


## Queries

Now try to do the original queries in TD1 on the FILMS dataset using **JSONPath**.

If you find it two hard to do a query on pure JSONPath, you can mix JSONPath and pure Python using the results of a JSONPath query. Nevertheless remember that this would not be directly applicable to other JSONPath implementations, specially in other languages. 

In [ ]:
query_title = "1. Get all the titles"

jsonpath_expr = parse('$.arrFilms[:].TITRE.title')

result = [match.value for match in jsonpath_expr.find(FILMS)]


print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")



In [ ]:
query_title = "2. Get all the titles of films from 1980"

year = 1980
jsonpath_expr = parse(f'$.arrFilms[?(@.Annee={year})].TITRE.title')

result = [match.value for match in jsonpath_expr.find(FILMS)]


print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "3. Alien abstract"



title = "Alien"
jsonpath_expr = parse(f'$.arrFilms[?(@.TITRE.title == "{title}")].RESUME')

result = [match.value for match in jsonpath_expr.find(FILMS)]


print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "4. Films with Bruce Willis"


name, surname = "Bruce", "Willis"
jsonpath_expr = parse(f'$.arrFilms[?(@.ROLES[?(PRENOM == "{name}" & NOM == "{surname}")])].TITRE.title')

result = [match.value for match in jsonpath_expr.find(FILMS)]


print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "5. Films with a RESUME"


tag = "RESUME"
jsonpath_expr = parse(f'$.arrFilms[?(@.{tag})].TITRE.title')

result = [match.value for match in jsonpath_expr.find(FILMS)]


print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "6. Films without a RESUME"

# This one might be a little harder than before, as negations are not implemented in this library



jsonpath_expr = parse(f'$.arrFilms[?(@.RESUME)].TITRE.title')
with_resume = [match.value for match in jsonpath_expr.find(FILMS)]

jsonpath_expr = parse(f'$.arrFilms[:].TITRE.title')
all_titles = [match.value for match in jsonpath_expr.find(FILMS)]

print(query_title)
print("\n" + "-"*30 + "\n")
print(set(all_titles).difference(set(with_resume)))
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "7. Films older than 30 years"

from datetime import date

year_dif = 30
year_now = date.today().year
jsonpath_expr = parse(f'$.arrFilms[?(@.Annee<{year_now-year_dif})].TITRE.title')
result = [match.value for match in jsonpath_expr.find(FILMS)]


print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "8. Role of Harvey Keitel in Reservoir Dogs"


jsonpath_expr = parse(f'$.arrFilms[?(@.TITRE.title =~ "Reservoir dogs")].ROLES[?(@.PRENOM == "Harvey" & @.NOM == "Keitel")].INTITULE')

result = [match.value for match in jsonpath_expr.find(FILMS)]

print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "9. Last film in database"


jsonpath_expr = parse(f'$.arrFilms[-1].TITRE.title')

result = [match.value for match in jsonpath_expr.find(FILMS)]

print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "10. Film preceding The Shining"


jsonpath_expr = parse(f'$.arrFilms[?(@.TITRE.title =~ "Shining")]')
film = [match for match in jsonpath_expr.find(FILMS)][0]
film_idx = film.path.index

jsonpath_expr = parse(f'$.arrFilms[:].TITRE.title')
all_titles = [match.value for match in jsonpath_expr.find(FILMS)]

result = all_titles[film_idx-1]

print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "11. Director of Vertigo"

jsonpath_expr_id_mes = parse('$.arrFilms[?( @.TITRE.title =~ "Vertigo") ].MES')
key_id = [match.value for match in jsonpath_expr_id_mes.find(FILMS)][0]

jsonpath_expr = parse(f'$.arrArtistes[?( @.id_art ==  {key_id})]')
result = [match.value for match in jsonpath_expr.find(FILMS)]

print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "12. Titles with an S"


jsonpath_expr = parse(f'$.arrFilms[?( @.TITRE.title =~  "S" )].TITRE.title')
result = [match.value for match in jsonpath_expr.find(FILMS)]

print(query_title)
print("\n" + "-"*30 + "\n")
print(result)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "13. Nodes with three descendants"


nbDesc = 3

jsonpath_expr = parse(f'$.*[?( @.`len` == {nbDesc} )]')
result = [match.value for match in jsonpath_expr.find(FILMS)]

print(query_title)
print("\n" + "-"*30 + "\n")
for r in result:
    print(r)
    print("*"*30)
print("\n" + "-"*30 + "\n")

In [ ]:
query_title = "14. Nodes whose name (tag) contains TU"

# This is again not easy to do. My solution does not work for some tags


pattern = "TU"

jsonpath_expr = parse('$..*')
result = [match.value for match in jsonpath_expr.find(FILMS)]

# Get everything, then "flatten"

dicts = [x for x in result if isinstance(x, dict)]
tosee = list(filter(lambda r: isinstance(r, list), result))

while tosee:
    a = tosee.pop(0)
    if isinstance(a, dict):
        dicts.append(a)
    elif isinstance(a, list):
        tosee.extend(a)

final = []
        
for d in dicts:
    for k, v in d.items():
        if pattern in k:
            final.append({k:v})
        
        
        
print(query_title)
print("\n" + "-"*30 + "\n")
for r,_ in zip(final, range(10)):
    print(r)
    
print("\n" + "-"*30 + "\n")